In [46]:
#this is to digitize lithologies for better processing. It will be easier to plot them
def str_array2floats(strarray):
    floats=[]
    for s in strarray:
        try:
            floats.append(float(s))
        except:
            floats.append(np.nan)
    return floats


# np.array(lith_labels).shape
def merge_similar_layers(loc_layer,loc_lith_labels):
    res_layer=[]
    res_label=[]
    for a in np.unique(loc_lith_labels):
        single_layer_prop=0
        if a!=-1:
            single_layer_prop=0
            for n,l in zip(loc_lith_labels,loc_layer):
                if a==n:
                    single_layer_prop += l
            res_layer.append(single_layer_prop)
            res_label.append(a)
    return res_layer,res_label

# for processing purpose at all locations, same number of layers and labels prepared. 
def make_same_numb_layers(master_l_thicknesses,lith_labels):
    layersinlocations=[len(l) for l in master_l_thicknesses]
    maxls=max(layersinlocations)
    j=-1
    for ml in master_l_thicknesses:
        j +=1
        if len(ml)<maxls:
            for i in range(maxls-len(ml)):
                ml.append(0)
                lith_labels[j]=np.append(lith_labels[j],-1)
    loc_layers=np.array(master_l_thicknesses)
    return loc_layers,np.array(lith_labels)

#it is thought that if all the locations have layer thickness information according to sorted layers, it will be lot easier to 
#plot or process the information so the function for sorting single location and creating zero thickness layer for missing 
#layer
def get_sortedlayers4loc(mod_loc_layers,mod_loc_layer_lbls,unique_lbls):
    sorted_loc_layer=[]
    for ul in unique_lbls:
        sorted_loc_layer.append(0)
        for e,l in zip(mod_loc_layers,mod_loc_layer_lbls):
            if l==ul:
                sorted_loc_layer[-1]=e
#     sorted_loc_layer[-1]
    return sorted_loc_layer
def get_depths(thickness_layer_array,elevation):
    depths=[-elevation]
    d=-elevation   
    for e in thickness_layer_array:
        d += e
        depths.append(d)
    return depths

In [47]:
# Modules and data loading
import numpy as np
import pandas as pd
import pickle
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import re
import math
from scipy.interpolate import griddata,interp2d
# import mpl_toolkits.mplot3d.art3d.Poly3DCollection
from   mpl_toolkits.mplot3d.art3d import Poly3DCollection
sys.path.append('..')
from pickle_fns import *
base_folder=r'D:\Ameyem Office\Projects\Electric surveys\Easwar files\jalaun\\'
base_folder=r'D:\Ameyem Office\Projects\Electric surveys\Easwar files\Mahoba\\'
vesdf,data_dfs=load_pkl(base_folder+'dvesdf_datadf.pkl')
# vesdf,data_dfs=load_pkl('tikamgarh.pkl')
E,N,Ele=vesdf.Easting.values.astype(np.float),vesdf.Northing.values.astype(np.float),str_array2floats(vesdf.RL.values)
# Ele=[float(re.findall('\d*\.\d+|\d+',E)[0]) for E in Ele]
# E,N,data_dfs[0]

nlayers=max([len(d) for d in data_dfs])
all_liths=[[]]*nlayers
all_liths=[]
for i in range(nlayers):
    all_liths.append([])
for df in data_dfs:
    for i in range(len(df['Interpreted Lithology'])):

        all_liths[i].append(df['Interpreted Lithology'].iloc[i])
#         break
#     break
for a in all_liths :
    print(np.unique(a))

['Kankar mix sand' 'Soil with clay' 'Soil with kankar ' 'alluvim boil'
 'black soil with kankar ' 'clay with  kankar' 'clay with kankar'
 'clay with kankar ' 'top soil  with clay' 'top soil with clay'
 'top soil with kankar' 'top soil with sticky clay' 'yellowish sandy clay']
['Course sand' 'Weathered pink granite' 'clay with quartz'
 'clay with weathered quartz' 'pink granite massive mineral increase '
 'soil with weathered granite' 'sticky clay with quartz '
 'sticky yellowish clay with quartz' 'weathered gray granite'
 'weathered pink granite' 'weathered pink granite medium grain']
[' pink granite little black fine chips' 'Weathered pink granite'
 'dolarite fine grain' 'grey granite  mafic mineral increase'
 'm fine to coarse green sand' 'pink granite fine grain size'
 'pink granite fine to medium grain' 'pink granite little black'
 'pink granite slightly black' 'pink whitish granite medium grain'
 'pinkgranite medium grain' 'pinkish granite medium grain'
 'weathered course size pin

In [48]:
def find_not_in_lithDict(all_unique_liths,lith_dict):
    all_lith=[]
    for j in lith_dict:
        all_lith.extend(lith_dict[j] )
    to_be_included=[]
    for l in all_unique_liths:
        if not l in all_lith:
            to_be_included.append(l)
    return to_be_included
def get_lithdict():
    top_soil_kankars=[
            'Course sand',
            'Kankar mix sand', 
            'Soil with clay', 'Soil with kankar ', 
            'alluvim boil', 'black soil with kankar ', 'clay with  kankar',
            'clay with kankar', 'clay with kankar ', 

     'soil with weathered granite',
     'sticky clay with quartz ',
     'sticky yellowish clay with quartz',
     'top soil  with clay',
     'top soil with clay',
     'top soil with kankar',
     'top soil with sticky clay',

     'yellowish sandy clay'  ]
#     clay_with_quartz=['clay with quartz',
#         'clay with weathered quartz',
#     ]
    weathered_granite=[
        'clay with quartz',
        'clay with weathered quartz',
        'weathered course size pink granite',
            'weathered gray granite', 'weathered pink granite',
            'weathered pink granite and touldes formation',
            'weathered pink granite medium grain',
            'weathered pink granite slightly green','Weathered pink granite',
                      ]
    hard_granite=['quartz little black','whitish pink granite coasse grain little black','pink granite boulder couse grain',
     'pink granite fine chips',
     'pink granite fine grain',
     'pink granite fine grain size',
     'pink granite fine to medium grain',
     'pink granite hard and compect',
     'pink granite hard compact',
     'pink granite in mafice mineral increase',
     'pink granite little black',
     'pink granite little black medium chips',
     'pink granite loose formation',
     'pink granite mafic mineral increase',
     'pink granite mafic mineral increased ',
     'pink granite mafic mineral increse',
     'pink granite marfic mineral increase',
     'pink granite massive mineral increase ',
     'pink granite massive mineral increse',
     'pink granite mud to fine graines black',' pink granite little black fine chips',
                                'Pink granite compressively less mafic','pink granite slightly black',
            'pink granite slightly black ,white fine formation',
            'pink granite slightly black ,whitish causse grain',
            'pink granite slightly black fine grain',
            'pink granite slightly black fine to medium',
            'pink granite slightly black fine to mud',
            'pink granite slightly black granite',
            'pink granite slightly black mud chip',
            'pink granite slightly black, white fine to medium',
            'pink granite slightly black, whitish cuasse',
            'pink granite very fine grain', 'pink granite very hard',
            'pink granite with coarse to fine grain',
            'pink granite with fine grain', 'pink granite(hard and compact)',
            'pink graniteloss formation', 'pink whitish granite medium grain',
            'pinkgranite medium grain', 'pinkish granite medium grain',
             'pure pink granite',
            'pure pink granite fine to medium chips',
            'pure pink granite slightly white medium gas',
            'pure pink granite slightly white medium gas ','quartz and feldspar dominant pink granite',
   'dolarite fine grain',
            'dolarite fine to medium chip',
                  '20%pink and 80% grey granite', 'very hard and compact grey ','pure black granite fine grain ',
                   'gray granite marfic intrusion',
            'gray granite with marfic mineral increased',
            'grey granite  little pink course grain',
            'grey granite  mafic mineral increase',
            'grey granite (hard and compact)',
            'grey granite ,whitish coasse grain',
            'grey granite coarse gain chips',
            'grey granite finre to medium grain', 'grey granite harded compect',
            'grey granite mafic intrusion hard and compect',
            'grey granite mafic mineral increase',
            'grey granite mafic mineral increased',
            'grey granite marfic mineral increase', 'grey granite medium grain',
            'grey granite pink medium grain', 'grey granite slaightly pink',
            'grey granite slightly white coosse grain',
            'grey granite slightly whitish',
            'grey granite white ,slightly pink coasse grain',
            'grey granite whitish coasse grain',
            'grey granite with mafic increased',
            'grey whitish granite coarse grain',
            'hard gray granite with feldspar', 'm fine to coarse green sand',
            'marfic mineral incresed gray granite','very hard and compact grey ',
            'very hard gray  granite',
                 ]
    final_layer=[ 'Final Layer',]
    lith_dict={0:top_soil_kankars,1:weathered_granite,2:hard_granite,3:final_layer}
    return lith_dict
       


uall_liths=[np.unique(a) for a in all_liths]
[len(a) for a in all_liths],[len(a) for a in uall_liths],uall_liths
ualiths=[]
for u in uall_liths:
    ualiths.extend(u)
len(ualiths ),len(np.unique(ualiths) ),np.unique(ualiths) 

find_not_in_lithDict(np.unique(ualiths),get_lithdict()) 

[]

In [49]:


lith_dict_summay={
    0:'Soil/Dry soil/ Soil with kankars',
#     1:'Clayee/Weathered granite/Fractured granite',
    1:'Weathered granite',
#     3:'Weathered Granite to granite mafic mineral increase',
    2:'Hard massive granite'
}

In [50]:
lith_dict=get_lithdict()
save_pkl(base_folder+'lith_dict.pkl',lith_dict)

lith=[]
for df in data_dfs:
#     print(df.iloc[0])
    lith.extend(df['Interpreted Lithology'].values)
# data_dfs[0]
np.unique(np.array(lith))

array([' pink granite little black fine chips',
       '20%pink and 80% grey granite', 'Course sand', 'Final Layer',
       'Kankar mix sand', 'Pink granite compressively less mafic',
       'Soil with clay', 'Soil with kankar ', 'Weathered pink granite',
       'alluvim boil', 'black soil with kankar ', 'clay with  kankar',
       'clay with kankar', 'clay with kankar ', 'clay with quartz',
       'clay with weathered quartz', 'dolarite fine grain',
       'dolarite fine to medium chip', 'gray granite marfic intrusion',
       'gray granite with marfic mineral increased',
       'grey granite  little pink course grain',
       'grey granite  mafic mineral increase',
       'grey granite (hard and compact)',
       'grey granite ,whitish coasse grain',
       'grey granite coarse gain chips',
       'grey granite finre to medium grain', 'grey granite harded compect',
       'grey granite mafic intrusion hard and compect',
       'grey granite mafic mineral increase',
       'grey grani

In [51]:
def get_lables(df,lith_dict):
    labels=[]
    for s in df['Interpreted Lithology'].values:
        print(s)
        for i in range(len(lith_dict)):
            if s in lith_dict[i]:
#                 print(lith_dict[i])
                if i in labels and i!=labels[-1]:
                    labels.append(i*10)
                else:
                    labels.append(i)
                break
    return labels
# get_lables(df,lith_dict),df
#utilization of get labels
for df in data_dfs:
    print('start**************')
#     print(get_lables(df,lith_dict) )
    
    df['lith_label']=get_lables(df,lith_dict)  
    print(df['lith_label'].values)
    print('************end')
lith_labels=np.array(df['lith_label'].values)

start**************
Soil with clay
Course sand
Weathered pink granite
pink granite(hard and compact)
pink granite slightly black fine to medium
Pink granite compressively less mafic
Final Layer
[0 0 1 2 2 2 3]
************end
start**************
clay with kankar
Weathered pink granite
pink granite little black
pink granite massive mineral increse
pink graniteloss formation
pink granite massive mineral increse
pink granite loose formation
Final Layer
[0 1 2 2 2 2 2 3]
************end
start**************
top soil with sticky clay
sticky clay with quartz 
Weathered pink granite
pink granite slightly black granite
pink granite marfic mineral increase
hard gray granite with feldspar
marfic mineral incresed gray granite
pink granite mud to fine graines black
Final Layer
[0 0 1 2 2 2 2 2 3]
************end
start**************
clay with kankar 
clay with weathered quartz
weathered pink granite and touldes formation
pink granite mafic mineral increse
grey granite slightly white coosse grain
pin

In [52]:
for vv,df in zip(vesdf['VES No.'].values,data_dfs):
#     print(vv)
    if vv=='121':
        print(df)

In [53]:
# lith_labels

#construction of lith labels for layers or thicknesses

np.set_printoptions(precision=3, suppress=True)
master_l_thicknesses=[]
lith_labels=[]
liths=[]
max_strata_thickness=400
for df in data_dfs:
#     print(df['Thickness(m)'].values.astype(np.float))
    layers_values=str_array2floats(df['Thickness(m)'].values)
    lastlayer_thickness=[max_strata_thickness-np.nansum(layers_values) if math.isnan(x) else x for x in layers_values]
#     print(lastlayer_thickness)
    master_l_thicknesses.append(lastlayer_thickness)
    lith_labels.append(np.array(df['lith_label'].values))
    liths.append(np.array(df['Interpreted Lithology'].values))
elevations=str_array2floats(vesdf.RL.values)
# layers_values,np.nansum(layers_values),master_l_thicknesses,lith_labels
# with lith dict here most of the layers are repeated.
# And similar layers need to be merged
for mt,ll in zip(master_l_thicknesses,lith_labels):
    print(mt,'\t\t\t',ll)

[5.64, 5.94, 19.75, 19.75, 45.12, 124.04, 179.75999999999999] 			 [0 0 1 2 2 2 3]
[5.64, 6.25, 44.36, 95.88, 11.28, 33.84, 5.64, 197.11000000000001] 			 [0 1 2 2 2 2 2 3]
[11.13, 25.38, 22.56, 31.02, 45.12, 33.84, 16.22, 16.92, 197.81] 			 [0 0 1 2 2 2 2 2 3]
[11.13, 16.92, 7.31, 71.65, 33.84, 16.92, 11.28, 33.84, 197.10999999999996] 			 [0 1 1 2 2 2 2 2 3]
[5.64, 8.14, 64.23, 124.08, 197.91] 			 [0 1 2 2 3]
[11.13, 14.1, 8.46, 8.46, 42.3, 25.38, 22.56, 70.5, 197.10999999999999] 			 [0 0 1 2 2 2 2 2 3]
[8.31, 4.53, 3.93, 14.1, 115.62, 56.4, 197.10999999999999] 			 [0 2 2 2 2 2 3]
[5.64, 5.64, 12.64, 71.96, 78.96, 28.2, 196.96000000000004] 			 [0 1 1 2 2 2 3]
[12.19, 15.86, 174.84, 197.11000000000001] 			 [0 1 2 3]
[5.64, 5.64, 12.64, 71.96, 78.96, 28.2, 196.96000000000004] 			 [0 1 1 2 2 2 3]
[11.13, 15.27, 63.69, 56.4, 25.2, 28.2, 200.11000000000001] 			 [0 1 2 2 2 2 3]
[11.13, 11.28, 8.38, 48.02, 33.84, 4.28, 5.64, 5.64, 11.58, 56.4, 203.80999999999997] 			 [0 1 1 2 2 2 2 2 2 2 3]
[2

In [54]:
#utilization of merge_similar_layers  and make_same_numb_layers functions

mod_layers=[]
mod_layer_lbls=[]
# 
for lyr,lbl in zip(master_l_thicknesses,lith_labels): # changed from loc_layers,lith_labels
    mlyr,mlbl=merge_similar_layers(lyr,lbl)
    mod_layers.append(mlyr)
    mod_layer_lbls.append(mlbl)

for mt,ll in zip(mod_layers, mod_layer_lbls ):
    print(mt,'\t\t\t',ll)

[11.58, 19.75, 188.91000000000003, 179.75999999999999] 			 [0, 1, 2, 3]
[5.64, 6.25, 191.0, 197.11000000000001] 			 [0, 1, 2, 3]
[36.51, 22.56, 143.12, 197.81] 			 [0, 1, 2, 3]
[11.13, 24.23, 167.53, 197.10999999999996] 			 [0, 1, 2, 3]
[5.64, 8.14, 188.31, 197.91] 			 [0, 1, 2, 3]
[25.23, 8.46, 169.2, 197.10999999999999] 			 [0, 1, 2, 3]
[8.31, 194.58, 197.10999999999999] 			 [0, 2, 3]
[5.64, 18.28, 179.11999999999998, 196.96000000000004] 			 [0, 1, 2, 3]
[12.19, 15.86, 174.84, 197.11000000000001] 			 [0, 1, 2, 3]
[5.64, 18.28, 179.11999999999998, 196.96000000000004] 			 [0, 1, 2, 3]
[11.13, 15.27, 173.48999999999998, 200.11000000000001] 			 [0, 1, 2, 3]
[11.13, 19.66, 165.4, 203.80999999999997] 			 [0, 1, 2, 3]
[5.64, 6.61, 190.55, 197.19999999999999] 			 [0, 1, 2, 3]
[5.64, 33.349999999999994, 163.9, 197.11000000000001] 			 [0, 1, 2, 3]
[16.770000000000003, 186.12, 197.10999999999999] 			 [0, 2, 3]
[5.35, 9.65, 187.89000000000001, 197.10999999999999] 			 [0, 1, 2, 3]
[5.64, 5.88, 19

In [55]:
# final_layers,final_labels=segregate_layers(mod_layers,mod_layer_lbls)
# final_layers,final_labels=make_same_numb_layers(mod_layers,mod_layer_lbls)


#create zero thickness layers for missing layers in each location
mod_layers,mod_layer_lbls
def get_unique_labels(mod_layer_lbls):
    res=[]
    for m in mod_layer_lbls:
        res +=m
    unique_lbls=np.unique(res)
    return unique_lbls

unique_lbls=get_unique_labels(mod_layer_lbls)

final_thicks=[]
for m_llyrs,m_llbls in zip(mod_layers,mod_layer_lbls):
    final_thicks.append(get_sortedlayers4loc(m_llyrs,m_llbls,unique_lbls))
unique_lbls,final_thicks

(array([0, 1, 2, 3], dtype=int64),
 [[11.58, 19.75, 188.91000000000003, 179.75999999999999],
  [5.64, 6.25, 191.0, 197.11000000000001],
  [36.51, 22.56, 143.12, 197.81],
  [11.13, 24.23, 167.53, 197.10999999999996],
  [5.64, 8.14, 188.31, 197.91],
  [25.23, 8.46, 169.2, 197.10999999999999],
  [8.31, 0, 194.58, 197.10999999999999],
  [5.64, 18.28, 179.11999999999998, 196.96000000000004],
  [12.19, 15.86, 174.84, 197.11000000000001],
  [5.64, 18.28, 179.11999999999998, 196.96000000000004],
  [11.13, 15.27, 173.48999999999998, 200.11000000000001],
  [11.13, 19.66, 165.4, 203.80999999999997],
  [5.64, 6.61, 190.55, 197.19999999999999],
  [5.64, 33.349999999999994, 163.9, 197.11000000000001],
  [16.770000000000003, 0, 186.12, 197.10999999999999],
  [5.35, 9.65, 187.89000000000001, 197.10999999999999],
  [5.64, 5.88, 191.37, 197.11000000000001],
  [17.37, 5.08, 180.45999999999998, 197.08999999999997],
  [17.37, 6.23, 179.02, 197.38],
  [6.09, 11.59, 185.21, 197.11000000000001]])

In [56]:
#final depths with default 0 to 5 or 0-10 layers
final_depths=[]
for ft,el in zip(final_thicks,elevations):
    fd = get_depths(ft,el)
    final_depths.append(fd)
final_depths=np.array(final_depths)
final_thicks=np.array(final_thicks)
# final_depths, final_thicks,unique_lbls      

for ll, lt,fd in zip(lith_labels,liths,final_depths):
    print(ll)
    print(lt)
    print(fd)

[0 0 1 2 2 2 3]
['Soil with clay' 'Course sand' 'Weathered pink granite'
 'pink granite(hard and compact)'
 'pink granite slightly black fine to medium'
 'Pink granite compressively less mafic' 'Final Layer']
[-228.   -216.42 -196.67   -7.76  172.  ]
[0 1 2 2 2 2 2 3]
['clay with kankar' 'Weathered pink granite' 'pink granite little black'
 'pink granite massive mineral increse' 'pink graniteloss formation'
 'pink granite massive mineral increse' 'pink granite loose formation'
 'Final Layer']
[-171.   -165.36 -159.11   31.89  229.  ]
[0 0 1 2 2 2 2 2 3]
['top soil with sticky clay' 'sticky clay with quartz '
 'Weathered pink granite' 'pink granite slightly black granite'
 'pink granite marfic mineral increase' 'hard gray granite with feldspar'
 'marfic mineral incresed gray granite'
 'pink granite mud to fine graines black' 'Final Layer']
[-225.   -188.49 -165.93  -22.81  175.  ]
[0 1 1 2 2 2 2 2 3]
['clay with kankar ' 'clay with weathered quartz'
 'weathered pink granite and touldes 

In [57]:
# np.append(np.array(N),final_depths,axis=1)
final_depths.shape
N.shape=(N.shape[0],1)
E.shape=(E.shape[0],1)
final_depths.shape,N.shape
xy_final_depths=np.append(np.array(E),np.append(np.array(N),final_depths,axis=1),axis=1)
np.save(base_folder+'dfinal_depths_labes_summary.npy',(xy_final_depths,unique_lbls,lith_dict_summay))

In [45]:
unique_lbls

array([0, 1, 2, 3, 4], dtype=int64)